In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import json
import requests

In [7]:
def getMovieName(movieName):
    #
    # 영화 검색화면에서 첫번째 영화의 코드를 가져온다.
    # 
    
    # 영화 이름이 검색되지않으면 None return
    try:
        SEARCH_URL = "https://watcha.com/ko-KR/search?"
        params = {
                "query" : movieName,
            }
        resp = requests.get(SEARCH_URL, params = params)
        soup = BeautifulSoup(resp.content, "html.parser")
        
        return soup.find("li", class_ = "css-106b4k6-Self").a['href'].split("/")[-1]
    except:
        return None

In [8]:
def setURL(movieName, category):
    #
    # getMovieName()에서 받아온 코드를 이용하여 URL을 만들어준다.
    # WATCHA_URL: overview를 위한 URL
    # WATCHA_API_URL: api를 사용하여 댓글을 가져오기 위한 URL
    #
    WATCHA_URL = "https://watcha.com/ko-KR/contents"
    WATCHA_API_URL = "https://api.watcha.com/api/contents"
    
    # 영화 이름이 검색되지않아서 영화 코드를 받지못했다면 return None
    try:
        MOVIE_CODE = getMovieName(movieName)
    except:
        return None
    
    # 카테고리 분류
    categoryDic = {
        'overview' : 'overview',
        'comments' : 'comments'    
    }
    
    # 헤더
    headers = {
        'Referer' : f"https://watcha.com/ko-KR/contents/{MOVIE_CODE}/{categoryDic[category]}",
        'x-watcha-client' : 'watcha-WebApp',
        'x-watcha-client-language' : 'ko',
        'x-watcha-client-region' : 'KR',
        'x-watcha-client-version' : "1.0.0"
    }

    requestList =[ WATCHA_URL, MOVIE_CODE, categoryDic[category] ]
    apiList = [ WATCHA_API_URL, MOVIE_CODE, categoryDic[category] ]

    REQUEST_URL = "/".join(requestList)
    API_URL = "/".join(apiList)
    
    print("movie code 변환 완료")
    
    # 입력받은 category가 'comments'인지 'overview'인지 확인 후 return
    return (API_URL, headers) if(category =='comments') else (REQUEST_URL, headers)

In [9]:
def getReviewList(movieName):
    #
    # 리뷰 리스트를 가져오는 함수
    # 영화가 존재하지않을때 return None
    #
    
    try:
        API_URL, headers = setURL(movieName, 'comments')
    except:
        return None
    print("API_URL 세팅 완료")
    
    # 파라미터, api를 통해 리뷰를 가져오기위해 요청할때 같이 보내는 파라미터
    # 기존에는 스크롤을 할때 리뷰가 3개씩 추가
    params = {
        'filter' : 'all',
        'order' : 'popular',
        'page' : 1,
        'size' : 20
    }

    # 페이로드 
    # 로그인 정보, 바꿔야함
    datas = {
        "email" : "yoooong2@gmail.com",
        "password" : "ABCD1234"
    }
    
    # return할 결과 list 
    reviewList = []
    
    # review 갯수 
    cntReview = 0
    
    # 로그인 정보
    session = requests.session()
    session.post("https://api.watcha.com/api/sessions", headers = headers, data = datas)

    # api로 받은 값을 list에 저장
    print("review crawling 시작")
    while(True):
        resp = session.get(API_URL, headers = headers, params = params)
        reviewJson = resp.json()
        if(len(reviewJson['result']['result']) == 0):
            break
        for i in reviewJson['result']['result']:
            reviewList.append(i)
            cntReview += 1
        params['page'] += 1
        
        # 100개씩 cnt
        if(cntReview%100) == 0:
            print(f"{cntReview}개")
        
    print(f"review {len(reviewList)}개 crawling 완료")
    return reviewList

In [10]:
def getOverview(movieName):
    #
    # 영화 정보를 가져오는 함수
    # 영화가 존재하지않을때 return None
    #
    
    try:
        REQUEST_URL, headers = setURL(movieName, 'overview')
    except:
        return None
    print("REQUEST_URL 세팅 완료")
    
    # return할 결과 dict
    overviewDict = {}
    
    resp = requests.get(REQUEST_URL, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")
    ulContents = soup.find("ul", class_ = "css-1w7dpv-VisualUl-DescriptionUl")
#     ulContents.select("dl dt")
#     ulContents.select("dl dd")


    # 원제, 제작 연도, 국가, 장르, 상영시간, 내용
    print("overview crawling 시작")
    for i in range(len(ulContents.select("dl dt"))):
        key = ulContents.select("dl dt")[i].text
        val = ulContents.select("dl dd")[i].text
        overviewDict[key] = val if(key != '제작 연도') else int(val)
    print("overview crawling 종료")
    
    return overviewDict

In [11]:
movieName = input("영화 제목을 입력하세요.\n")
output = getReviewList(movieName)
try:
    output.insert(0, getOverview(movieName))
except:
    print("영화를 찾지 못했습니다.\n올바른 영화이름을 입력해주세요.")

영화 제목을 입력하세요.
알라딘
movie code 변환 완료
API_URL 세팅 완료
review crawling 시작
100개
200개
300개
400개
500개
600개
700개
800개
900개
1000개
1100개
1200개
1300개
review 1312개 crawling 완료
movie code 변환 완료
REQUEST_URL 세팅 완료
overview crawling 시작
overview crawling 종료


In [12]:
pprint(output)


[{'국가': '미국',
  '내용': '원숭이 아부와 함께 살아가는 아그라바의 좀도둑 알라딘은 어느 날 궁궐을 탈출한 술탄의 딸 자스민에게 한눈에 반한다. 술탄의 '
        '자리를 노리는 사악한 마법사 자파는 알라딘을 눈여겨보고 그에게 마법의 램프를 가져다주면 부자로 만들어 주겠다고 말한다. '
        '동굴에 들어가서 램프를 찾던 알라딘은 도중에 동굴이 무너지는 바람에 갇히게 된다.  위기에 빠진 알라딘은 램프의 요정 지니와 '
        '마법 양탄자의 도움으로 동굴에서 탈출한다. 지니의 주인이 된 알라딘은 지니를 램프에서 구해주겠다고 약속하고, 지니의 힘을 '
        '빌어 왕자로 변신해 자스민에게 청혼하기 위해 아그라바로 돌아오는데...',
  '상영시간': '1시간 30분',
  '원제': 'Aladdin',
  '장르': '애니메이션/모험/코미디/가족/판타지/로맨스/키즈',
  '제작 연도': 1992},
 {'code': 'droQ06jyA2yNJ',
  'content_code': 'mJOV8Z5',
  'created_at': '2013-10-05T06:20:01+09:00',
  'current_context': {'comment_code': 'droQ06jyA2yNJ',
                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 201,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': 'A whole new world- \n이거 하나만으로도 별 다섯개 만점!',
  'user': {'code': '6nkPvr4rxgarB',
           'name': '행행',
           'official_use

                          'watched_at': None},
  'watched_at': None},
 {'code': 'WzO2gj89XEXZY',
  'content_code': 'mJOV8Z5',
  'created_at': '2014-11-03T08:31:27+09:00',
  'current_context': {'comment_code': 'WzO2gj89XEXZY',
                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 21,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': '지니=5점',
  'user': {'code': '8BRvep7aQq6VJ',
           'name': 'Bonnie',
           'official_user': False,
           'photo': {'large': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_300,w_300/v1539076208/gzjybj5ix1jwwuhugm6e.jpg',
                     'original': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/v1539076208/gzjybj5ix1jwwuhugm6e.jpg',
                     'small': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_100,w_100/v1539076208/gzjybj5ix1jwwuhugm6e.jpg'},
       

                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 14,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': 'Genie, you’re free',
  'user': {'code': 'jae5WKypa51P3',
           'name': 'wonst',
           'official_user': False,
           'photo': {'large': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_300,w_300/v1559723388/wecr23bds45emhwdrj74.jpg',
                     'original': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/v1559723388/wecr23bds45emhwdrj74.jpg',
                     'small': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_100,w_100/v1559723388/wecr23bds45emhwdrj74.jpg'},
           'watcha_play_user': True},
  'user_code': 'jae5WKypa51P3',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rating': 10,
                          'status': None,
               

                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 10,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': '언젠가 내가 이야기를 쓴다면, 그 이야기를 꼭 디즈니 영화로도 탄생시키고 싶다',
  'user': {'code': '6ewxaOl7pvQ1m',
           'name': 'WHITE',
           'official_user': False,
           'photo': {'large': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_300,w_300/v1525275912/f2sxitdfihkr5agvxmci.jpg',
                     'original': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/v1525275912/f2sxitdfihkr5agvxmci.jpg',
                     'small': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_100,w_100/v1525275912/f2sxitdfihkr5agvxmci.jpg'},
           'watcha_play_user': True},
  'user_code': '6ewxaOl7pvQ1m',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rati

                     'original': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/v1501300403/zvhxvk7sxwdffl8zvwyf.jpg',
                     'small': 'https://dhgywazgeek0d.cloudfront.net/watcha/image/upload/c_fill,h_100,w_100/v1501300403/zvhxvk7sxwdffl8zvwyf.jpg'},
           'watcha_play_user': True},
  'user_code': 'dP8v6W40Y5WeJ',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rating': 9,
                          'status': None,
                          'user_code': 'dP8v6W40Y5WeJ',
                          'watched_at': None},
  'watched_at': None},
 {'code': '6GWQ1Kde32J50',
  'content_code': 'mJOV8Z5',
  'created_at': '2015-01-19T12:31:29+09:00',
  'current_context': {'comment_code': '6GWQ1Kde32J50',
                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 7,
  'replies_count': 0,
  'replyable

...

  'watched_at': None},
 {'code': '6PzQ4gxznMqVk',
  'content_code': 'mJOV8Z5',
  'created_at': '2014-03-01T05:20:30+09:00',
  'current_context': {'comment_code': '6PzQ4gxznMqVk',
                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 0,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': '다 큰 요즘도 나한테 지니가 있었다면...하고 생각한다',
  'user': {'code': 'Qgy51BE27xDjk',
           'name': '이재경',
           'official_user': False,
           'photo': {'large': 'https://d19zyxn02q3dj8.cloudfront.net/user/480437/large/1b2223262a3a224d6b6dcad0ba2dfcea141e50d3.jpg',
                     'small': 'https://d19zyxn02q3dj8.cloudfront.net/user/480437/small/1b2223262a3a224d6b6dcad0ba2dfcea141e50d3.jpg'},
           'watcha_play_user': False},
  'user_code': 'Qgy51BE27xDjk',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                     

  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rating': 8,
                          'status': None,
                          'user_code': 'ld0q0ZRgO56Xn',
                          'watched_at': None},
  'watched_at': None},
 {'code': 'XpeMJAXxV2Jn6',
  'content_code': 'mJOV8Z5',
  'created_at': '2013-12-18T07:15:26+09:00',
  'current_context': {'comment_code': 'XpeMJAXxV2Jn6',
                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 0,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': '굿~~~~좋아요',
  'user': {'code': '87Gv7Z2yVqE6o',
           'name': '신장원',
           'official_user': False,
           'photo': {'large': 'https://d3sz5r0rl9fxuc.cloudfront.net/assets/default/user/photo_file_name_large-bc8b334acec6a4e386249dedf9e763b5e6aff523fa85cc29211f22e6bed540df.jpg',
                 

  'user': {'code': 'JYMKqmw6m5loD',
           'name': '시끄러운 불꽃',
           'official_user': False,
           'photo': {'large': 'https://d19zyxn02q3dj8.cloudfront.net/user/82103/large/ef4dba0aef53e6dec77dcf57e0fb3df0789b2290.gif',
                     'small': 'https://d19zyxn02q3dj8.cloudfront.net/user/82103/small/ef4dba0aef53e6dec77dcf57e0fb3df0789b2290.gif'},
           'watcha_play_user': True},
  'user_code': 'JYMKqmw6m5loD',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rating': 6,
                          'status': None,
                          'user_code': 'JYMKqmw6m5loD',
                          'watched_at': None},
  'watched_at': None},
 {'code': 'lP0E8Jz8jMbxg',
  'content_code': 'mJOV8Z5',
  'created_at': '2013-10-09T09:30:27+09:00',
  'current_context': {'comment_code': 'lP0E8Jz8jMbxg',
                      'improper': False,
                      'liked': False,
                      'sp

                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 0,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': '지니!! 나의 소원은 소원을 삼천개만 더 이루어 달라는것...돈으로 신분위장, 경쟁자 제거, 사기결혼 하는 '
          '영화....목적을 향해 열정적으로 계획하고 실행하는 자파의 근성!!',
  'user': {'code': 'Mr95n0eljqZPG',
           'name': '늑대',
           'official_user': False,
           'photo': {'large': 'https://d19zyxn02q3dj8.cloudfront.net/user/206104/large/44f31d4b3bc38e73095aff3b3ff5eb4a3f665ca5.jpg',
                     'small': 'https://d19zyxn02q3dj8.cloudfront.net/user/206104/small/44f31d4b3bc38e73095aff3b3ff5eb4a3f665ca5.jpg'},
           'watcha_play_user': False},
  'user_code': 'Mr95n0eljqZPG',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rating': 6,
                          'status': None,
                          'user_code': 'Mr95n0eljqZPG',
                         

 {'code': '96PzQ4p7nQqVk',
  'content_code': 'mJOV8Z5',
  'created_at': '2013-05-10T08:53:51+09:00',
  'current_context': {'comment_code': '96PzQ4p7nQqVk',
                      'improper': False,
                      'liked': False,
                      'spoiler': False},
  'improper': False,
  'likes_count': 0,
  'replies_count': 0,
  'replyable': True,
  'spoiler': False,
  'text': '노래만 좋음... 캐릭터가 엉망이다 특히 알라딘이',
  'user': {'code': 'r6NW5Qnp7x1Yo',
           'name': 'LJ',
           'official_user': False,
           'photo': {'large': 'https://d19zyxn02q3dj8.cloudfront.net/user/21973/large/2a6174ca8a45d40ec4e89802a11d09ce4ae535dd.jpg',
                     'small': 'https://d19zyxn02q3dj8.cloudfront.net/user/21973/small/2a6174ca8a45d40ec4e89802a11d09ce4ae535dd.jpg'},
           'watcha_play_user': False},
  'user_code': 'r6NW5Qnp7x1Yo',
  'user_content_action': {'content_code': 'mJOV8Z5',
                          'mehed': False,
                          'rating': 4,
           

TypeError: list indices must be integers or slices, not str

In [18]:
with open(f"movieName.json", 'w', encoding='utf-8') as file :
        json.dump(output, file, ensure_ascii=False, indent='\t')